In [4]:
import numpy as np
import pandas as pd
import requests,json,os
from urllib.parse import quote 
import os,glob

In [5]:
# 관광지명, 도로명주소, lat, lng, 분류
df = pd.read_excel('data/서울시 관광 명소_ljh.XLSX', engine='openpyxl')

In [6]:
df.to_csv('data/main/temp_files/서울시 관광 명소_ljh.csv', index=False)
df = pd.read_csv('data/main/temp_files/서울시 관광 명소_ljh.csv', encoding='utf-8', sep=',')

In [9]:
df = df[(df['언어']) == 'ko']
# 도로명주소만 남기기
df.rename(columns= {'상호명':'관광지명', '신주소':'도로명주소'}, inplace=True)
df = df[['관광지명', '도로명주소']]
df.reset_index(drop=True,inplace=True)
df.nunique()

In [14]:
df.to_csv('data/서울_명소_명소.csv',index=False)
df= pd.read_csv('data/서울_명소_명소.csv', encoding='utf-8', sep=',')

In [15]:
df.isna().sum().sum()

0

In [17]:
# 도로명주소에서 우편번호 제거
addr_list = []
for i in df.index:
    addr1 = df.도로명주소[i].split(' ')[1:]
    df.도로명주소[i]= ' '.join(addr1)
    addr_list.append(df.도로명주소[i])
    # df.도로명주소[i] = df.도로명주소[i].split(' ')[1:]
addr_list

['서울 영등포구 63로 50 (여의도동, 63한화생명빌딩) ',
 '서울 종로구 사직로 161 (세종로, 경복궁) ',
 '서울특별시 동대문구 경희대로 26 자연사박물관',
 '서울특별시 종로구 세종대로 172 ',
 '서울 종로구 효자로 12 (세종로, 국립고궁박물관) ',
 '서울 종로구 송월길 52 (송월동, 국립기상박물관) ',
 '서울 종로구 삼청로 37 (세종로, 국립민속박물관) ',
 '서울 용산구 서빙고로 137 (국립중앙박물관) ',
 '서울 용산구 서빙고로 139 (용산동6가, 국립한글박물관) ',
 '서울 종로구 삼청로 30 (소격동) ',
 '서울특별시 영등포구 의사당대로 1  (여의도동) ',
 '서울 용산구 남산공원길 105 (용산동2가, YTN서울타워) ',
 '서울시 종로구 세종대로 198 (세종로) ',
 '서울 중구 세종대로 99 ',
 '서울 강남구 도산대로45길 20 (도산전시관) ',
 '서울 중구 을지로 281 (을지로7가, 2-1) 동대문역사공원 내 동대문역사관',
 '서울 송파구 올림픽로 240 (잠실동, 롯데월드) ',
 '서울 송파구 올림픽로 300 (신천동) ',
 '서울 용산구 이태원로55길 60-16 (한남동) ',
 '서울시 중구 을지로 66 한국외환은행 ',
 '서울 중구 명동길 74 (명동성당) ',
 '서울 송파구 올림픽로 424 (몽촌역사관) ',
 '서울 중구 통일로 1 (서울역 본옥) ',
 '서울 강남구 봉은사로 531 (봉은사) ',
 '서울 종로구 율곡로 283 (종로6가, 서울디자인지원센터) ',
 '서울특별시 중구 세종대로 110 (태평로1가) ',
 '서울 중구 세종대로 119 (태평로1가) ',
 '서울 중구 덕수궁길 61 (서소문동, 서울시립미술관) ',
 '서울 종로구 새문안로 55 (신문로2가, 서울역사박물관) ',
 '서울 종로구 새문안로 55 (신문로2가) ',
 '서울특별시 강남구 선릉로100길 1 (삼성동) ',
 '서울 강남구 도산대로 441 (청담동) ',
 '서울 

In [18]:
# 도로명주소에서 우편번호 제거
for i in df.index:
    addr1 = df.도로명주소[i].split(' ')[1:]
    df.도로명주소[i] = (' '.join(addr1))
df.도로명주소

0                영등포구 63로 50 (여의도동, 63한화생명빌딩) 
1                      종로구 사직로 161 (세종로, 경복궁) 
2                          동대문구 경희대로 26 자연사박물관
3                                종로구 세종대로 172 
4                   종로구 효자로 12 (세종로, 국립고궁박물관) 
5                   종로구 송월길 52 (송월동, 국립기상박물관) 
6                   종로구 삼청로 37 (세종로, 국립민속박물관) 
7                      용산구 서빙고로 137 (국립중앙박물관) 
8               용산구 서빙고로 139 (용산동6가, 국립한글박물관) 
9                            종로구 삼청로 30 (소격동) 
10                       영등포구 의사당대로 1  (여의도동) 
11             용산구 남산공원길 105 (용산동2가, YTN서울타워) 
12                         종로구 세종대로 198 (세종로) 
13                                 중구 세종대로 99 
14                     강남구 도산대로45길 20 (도산전시관) 
15    중구 을지로 281 (을지로7가, 2-1) 동대문역사공원 내 동대문역사관
16                   송파구 올림픽로 240 (잠실동, 롯데월드) 
17                         송파구 올림픽로 300 (신천동) 
18                    용산구 이태원로55길 60-16 (한남동) 
19                           중구 을지로 66 한국외환은행 
20                           중구 명동길 74 (명동성당) 
21           

In [20]:
# 2 kakao API로 도로명 주소 구하는 함수
def kakao_location(place):
    with open('data/main/key/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [21]:
# 2
temp1 =[]
for i in df.index:
    try:
        # target = df['관광지명'][i].strip()
        temp1.append(kakao_location(df.도로명주소.values[i]))
    except:
        print(i, df.관광지명[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df2 = pd.concat([df, df_test], axis=1)

32 송파관광정보센터


In [30]:
df2.lat.isna().sum()

1

In [39]:
df2.tail()

,관광지명,도로명주소,lat,lng
44,종묘,"종로구 종로 157 (훈정동, 종묘공원)",37.577624,126.993841
45,창경궁,"종로구 창경궁로 185 (와룡동, 창경궁)",37.577624,126.993841
46,창경궁 대온실,"종로구 창경궁로 185 (와룡동, 창경궁)",37.578870,126.989710
47,창덕궁,"종로구 율곡로 99 (와룡동, 창덕궁)",37.515429,127.120801
48,한성백제박물관,"송파구 위례성대로 71 (방이동, 한성백제박물관)",NaN,NaN


In [35]:
df3 = df2[df2['lat'].notnull()]
df3.tail()

,관광지명,도로명주소,lat,lng
43,조계사,종로구 우정국로 55 (조계사),37.573134,126.994864
44,종묘,"종로구 종로 157 (훈정동, 종묘공원)",37.577624,126.993841
45,창경궁,"종로구 창경궁로 185 (와룡동, 창경궁)",37.577624,126.993841
46,창경궁 대온실,"종로구 창경궁로 185 (와룡동, 창경궁)",37.578870,126.989710
47,창덕궁,"종로구 율곡로 99 (와룡동, 창덕궁)",37.515429,127.120801


In [38]:
# 4 처리 끝난 파일 저장
df3.to_csv('data/서울_명소_명소_final.csv',index=False)
# 체크
df4 = pd.read_csv('data/서울_명소_명소_final.csv', encoding='utf-8', sep=',')
df4.head()

,관광지명,도로명주소,lat,lng
0,63스퀘어,"영등포구 63로 50 (여의도동, 63한화생명빌딩)",37.519664,126.939909
1,경복궁,"종로구 사직로 161 (세종로, 경복궁)",37.575904,126.976842
2,경희대학교 자연사박물관,동대문구 경희대로 26 자연사박물관,37.593949,127.054891
3,광화문광장,종로구 세종대로 172,37.571623,126.976788
4,국립고궁박물관,"종로구 효자로 12 (세종로, 국립고궁박물관)",37.576592,126.974975
